In [1]:
import findspark
findspark.init()

#Local standalone mode; exekutoren;wie paralleliziert spark


In [2]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("BigData").getOrCreate()
#Quelle: https://sparkbyexamples.com/pyspark/pyspark-parallelize-create-rdd/
sc = spark.sparkContext
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/13 19:11:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Speicherort der Stationsinformationen festlegen
folder_Stations = '/Users/danielwentsch/Desktop/BigData/Data/01_Stations/'
#Speicherort der Temperaturdaten
folder_Temperature = '/Users/danielwentsch/Desktop/BigData/Data/02_air_temperature/'
#Datenbeschreibung: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/air_temperature/historical/BESCHREIBUNG_obsgermany_climate_10min_tu_historical_de.pdf 
#Speicherort der Winddaten
folder_Wind = '/Users/danielwentsch/Desktop/BigData/Data/03_wind/'
#Speicherort der Sonnendaten
folder_Solar = '/Users/danielwentsch/Desktop/BigData/Data/04_solar/'
#Datenbeschreibung: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/solar/historical/BESCHREIBUNG_obsgermany_climate_10min_solar_historical_de.pdf

In [4]:
#Loading Temperature Data in df
#Quelle = https://stackoverflow.com/questions/49471192/spark-2-3-0-read-text-file-with-header-option-not-working
df_Temperatur = spark.read.option("header", "true").option("delimiter", ";").option("inferSchema", "false").csv(folder_Temperature+'*.txt')


In [5]:
#Anzahl der Datensätze im DataFrame Temperature
df_Temperatur.count()


10977397

In [6]:
#Struktur der DF Temperature
df_Temperatur.show(3,False)

+-----------+------------+-----+-------+------+------+------+------+---+
|STATIONS_ID|MESS_DATUM  |  QN |PP_10  |TT_10 |TM5_10|RF_10 |TD_10 |eor|
+-----------+------------+-----+-------+------+------+------+------+---+
|       2171|201001010000|    3|  960.8|  -1.3|  -1.3|  94.5|  -2.1|eor|
|       2171|201001010010|    3|  960.8|  -1.4|  -1.3|  94.6|  -2.2|eor|
|       2171|201001010020|    3|  960.7|  -1.4|  -1.3|  94.9|  -2.1|eor|
+-----------+------------+-----+-------+------+------+------+------+---+
only showing top 3 rows



In [7]:
#Schema des DF anzeigen
df_Temperatur.printSchema()

root
 |-- STATIONS_ID: string (nullable = true)
 |-- MESS_DATUM: string (nullable = true)
 |--   QN: string (nullable = true)
 |-- PP_10: string (nullable = true)
 |-- TT_10: string (nullable = true)
 |-- TM5_10: string (nullable = true)
 |-- RF_10: string (nullable = true)
 |-- TD_10: string (nullable = true)
 |-- eor: string (nullable = true)



In [8]:
from pyspark.sql.types import *
df_Temperatur = df_Temperatur.withColumn('PP_10',df_Temperatur.PP_10.cast(FloatType()))
df_Temperatur = df_Temperatur.withColumn('STATIONS_ID',df_Temperatur.STATIONS_ID.cast(IntegerType()))
df_Temperatur.printSchema()

root
 |-- STATIONS_ID: integer (nullable = true)
 |-- MESS_DATUM: string (nullable = true)
 |--   QN: string (nullable = true)
 |-- PP_10: float (nullable = true)
 |-- TT_10: string (nullable = true)
 |-- TM5_10: string (nullable = true)
 |-- RF_10: string (nullable = true)
 |-- TD_10: string (nullable = true)
 |-- eor: string (nullable = true)



In [9]:
#Aus MESS_DATUM Jahr,Monat und Tag auslesen
#Quelle: https://www.datasciencemadesimple.com/extract-first-n-and-last-n-character-in-pyspark/
df_Temperatur  = df_Temperatur.withColumn("Jahr", df_Temperatur.MESS_DATUM.substr(1,4))
df_Temperatur  = df_Temperatur.withColumn("Monat", df_Temperatur.MESS_DATUM.substr(5,2))
df_Temperatur  = df_Temperatur.withColumn("Tag", df_Temperatur.MESS_DATUM.substr(7,2))
df_Temperatur  = df_Temperatur.withColumn("Stunde", df_Temperatur.MESS_DATUM.substr(9,2))
df_Temperatur  = df_Temperatur.withColumn("Minute", df_Temperatur.MESS_DATUM.substr(11,2))
df_Temperatur.show(4)

+-----------+------------+-----+-----+------+------+------+------+---+----+-----+---+------+------+
|STATIONS_ID|  MESS_DATUM|   QN|PP_10| TT_10|TM5_10| RF_10| TD_10|eor|Jahr|Monat|Tag|Stunde|Minute|
+-----------+------------+-----+-----+------+------+------+------+---+----+-----+---+------+------+
|       2171|201001010000|    3|960.8|  -1.3|  -1.3|  94.5|  -2.1|eor|2010|   01| 01|    00|    00|
|       2171|201001010010|    3|960.8|  -1.4|  -1.3|  94.6|  -2.2|eor|2010|   01| 01|    00|    10|
|       2171|201001010020|    3|960.7|  -1.4|  -1.3|  94.9|  -2.1|eor|2010|   01| 01|    00|    20|
|       2171|201001010030|    3|960.8|  -1.4|  -1.3|  95.1|  -2.1|eor|2010|   01| 01|    00|    30|
+-----------+------------+-----+-----+------+------+------+------+---+----+-----+---+------+------+
only showing top 4 rows



In [10]:
#Neue IDSpalte erstellen
#Quelle: https://sparkbyexamples.com/pyspark/pyspark-add-new-column-to-dataframe/#add%20column%20based%20on%20another%20column%20of%20dataframe
df_Temperatur = df_Temperatur.withColumn("ID",concat_ws('','STATIONS_ID','MESS_DATUM'))
df_Temperatur.show(4,False)

+-----------+------------+-----+-----+------+------+------+------+---+----+-----+---+------+------+----------------+
|STATIONS_ID|MESS_DATUM  |  QN |PP_10|TT_10 |TM5_10|RF_10 |TD_10 |eor|Jahr|Monat|Tag|Stunde|Minute|ID              |
+-----------+------------+-----+-----+------+------+------+------+---+----+-----+---+------+------+----------------+
|2171       |201001010000|    3|960.8|  -1.3|  -1.3|  94.5|  -2.1|eor|2010|01   |01 |00    |00    |2171201001010000|
|2171       |201001010010|    3|960.8|  -1.4|  -1.3|  94.6|  -2.2|eor|2010|01   |01 |00    |10    |2171201001010010|
|2171       |201001010020|    3|960.7|  -1.4|  -1.3|  94.9|  -2.1|eor|2010|01   |01 |00    |20    |2171201001010020|
|2171       |201001010030|    3|960.8|  -1.4|  -1.3|  95.1|  -2.1|eor|2010|01   |01 |00    |30    |2171201001010030|
+-----------+------------+-----+-----+------+------+------+------+---+----+-----+---+------+------+----------------+
only showing top 4 rows



In [11]:
#Unnötige Spalten droppen
#Quelle: https://sparkbyexamples.com/pyspark/pyspark-drop-column-from-dataframe/
#Spalte TT_10 und TM5_10 umbenennen
#Quelle: https://sparkbyexamples.com/pyspark/pyspark-rename-dataframe-column/
df_Temperatur = df_Temperatur.drop('  QN','PP_10','RF_10','TD_10','eor').withColumnRenamed('TT_10','Temp_2m').withColumnRenamed('TM5_10','Temp_5cm')

In [12]:
df_Temperatur.printSchema()

root
 |-- STATIONS_ID: integer (nullable = true)
 |-- MESS_DATUM: string (nullable = true)
 |-- Temp_2m: string (nullable = true)
 |-- Temp_5cm: string (nullable = true)
 |-- Jahr: string (nullable = true)
 |-- Monat: string (nullable = true)
 |-- Tag: string (nullable = true)
 |-- Stunde: string (nullable = true)
 |-- Minute: string (nullable = true)
 |-- ID: string (nullable = false)



In [13]:
#Loading SolarData in DF 
df_Solar = spark.read.option("header", "true").option("delimiter", ";").option("inferSchema", "false").csv(folder_Solar+'*.txt')

In [14]:
df_Solar.count()

29337980

In [15]:
#Struktur der DF Solar
df_Solar.show(3,False)

+-----------+------------+-----+------+------+--------+-----+---+
|STATIONS_ID|MESS_DATUM  |  QN |DS_10 |GS_10 |SD_10   |LS_10|eor|
+-----------+------------+-----+------+------+--------+-----+---+
|         73|201001010000|    3|  -999|  -999|   0.000|-999 |eor|
|         73|201001010010|    3|  -999|  -999|   0.000|-999 |eor|
|         73|201001010020|    3|  -999|  -999|   0.000|-999 |eor|
+-----------+------------+-----+------+------+--------+-----+---+
only showing top 3 rows



In [16]:
#Schema des DF anzeigen
df_Solar.printSchema()

root
 |-- STATIONS_ID: string (nullable = true)
 |-- MESS_DATUM: string (nullable = true)
 |--   QN: string (nullable = true)
 |-- DS_10: string (nullable = true)
 |-- GS_10: string (nullable = true)
 |-- SD_10: string (nullable = true)
 |-- LS_10: string (nullable = true)
 |-- eor: string (nullable = true)



In [17]:

df_Solar = df_Solar.withColumn('SD_10',df_Solar.SD_10.cast(FloatType()))
df_Solar = df_Solar.withColumn('STATIONS_ID',df_Solar.STATIONS_ID.cast(IntegerType()))
df_Solar.printSchema()

root
 |-- STATIONS_ID: integer (nullable = true)
 |-- MESS_DATUM: string (nullable = true)
 |--   QN: string (nullable = true)
 |-- DS_10: string (nullable = true)
 |-- GS_10: string (nullable = true)
 |-- SD_10: float (nullable = true)
 |-- LS_10: string (nullable = true)
 |-- eor: string (nullable = true)



In [18]:
df_Solar  = df_Solar.withColumn("Jahr", df_Solar.MESS_DATUM.substr(1,4))
df_Solar  = df_Solar.withColumn("Monat", df_Solar.MESS_DATUM.substr(5,2))
df_Solar  = df_Solar.withColumn("Tag", df_Solar.MESS_DATUM.substr(7,2))
df_Solar  = df_Solar.withColumn("Stunde", df_Solar.MESS_DATUM.substr(9,2))
df_Solar  = df_Solar.withColumn("Minute", df_Solar.MESS_DATUM.substr(11,2))
df_Solar.show(4)

+-----------+------------+-----+------+------+-----+-----+---+----+-----+---+------+------+
|STATIONS_ID|  MESS_DATUM|   QN| DS_10| GS_10|SD_10|LS_10|eor|Jahr|Monat|Tag|Stunde|Minute|
+-----------+------------+-----+------+------+-----+-----+---+----+-----+---+------+------+
|         73|201001010000|    3|  -999|  -999|  0.0| -999|eor|2010|   01| 01|    00|    00|
|         73|201001010010|    3|  -999|  -999|  0.0| -999|eor|2010|   01| 01|    00|    10|
|         73|201001010020|    3|  -999|  -999|  0.0| -999|eor|2010|   01| 01|    00|    20|
|         73|201001010030|    3|  -999|  -999|  0.0| -999|eor|2010|   01| 01|    00|    30|
+-----------+------------+-----+------+------+-----+-----+---+----+-----+---+------+------+
only showing top 4 rows



In [19]:
#Neue IDSpalte erstellen
df_Solar = df_Solar.withColumn("ID",concat_ws('','STATIONS_ID','MESS_DATUM'))
df_Solar.show(4,False)

+-----------+------------+-----+------+------+-----+-----+---+----+-----+---+------+------+--------------+
|STATIONS_ID|MESS_DATUM  |  QN |DS_10 |GS_10 |SD_10|LS_10|eor|Jahr|Monat|Tag|Stunde|Minute|ID            |
+-----------+------------+-----+------+------+-----+-----+---+----+-----+---+------+------+--------------+
|73         |201001010000|    3|  -999|  -999|0.0  |-999 |eor|2010|01   |01 |00    |00    |73201001010000|
|73         |201001010010|    3|  -999|  -999|0.0  |-999 |eor|2010|01   |01 |00    |10    |73201001010010|
|73         |201001010020|    3|  -999|  -999|0.0  |-999 |eor|2010|01   |01 |00    |20    |73201001010020|
|73         |201001010030|    3|  -999|  -999|0.0  |-999 |eor|2010|01   |01 |00    |30    |73201001010030|
+-----------+------------+-----+------+------+-----+-----+---+----+-----+---+------+------+--------------+
only showing top 4 rows



In [20]:
df_Solar = df_Solar.drop('  QN','DS_10','GS_10','LS_10','eor').withColumnRenamed('SD_10','Sum_Sonnenscheindauer')
df_Solar.printSchema()

root
 |-- STATIONS_ID: integer (nullable = true)
 |-- MESS_DATUM: string (nullable = true)
 |-- Sum_Sonnenscheindauer: float (nullable = true)
 |-- Jahr: string (nullable = true)
 |-- Monat: string (nullable = true)
 |-- Tag: string (nullable = true)
 |-- Stunde: string (nullable = true)
 |-- Minute: string (nullable = true)
 |-- ID: string (nullable = false)



In [21]:
#Für Hell und weiß im Januar --> Sommer heiß 
#Welche Daten brauch ich ? --> Temperatur des Sommers (Definiton Sommer?) + Sonnenstunden Januar + Schnee im Januar?
#Finaler Datensatz = Gruppiert nach Jahr und Stations ID die Temperatur im Sommer und die Sonnenstunden im Januar

In [22]:
#Bearbeiten der Datensätze, df_Solar und df_Temperature, damit nur die benötigten Informationen übrig sind. 
df_Solar.show(3)

+-----------+------------+---------------------+----+-----+---+------+------+--------------+
|STATIONS_ID|  MESS_DATUM|Sum_Sonnenscheindauer|Jahr|Monat|Tag|Stunde|Minute|            ID|
+-----------+------------+---------------------+----+-----+---+------+------+--------------+
|         73|201001010000|                  0.0|2010|   01| 01|    00|    00|73201001010000|
|         73|201001010010|                  0.0|2010|   01| 01|    00|    10|73201001010010|
|         73|201001010020|                  0.0|2010|   01| 01|    00|    20|73201001010020|
+-----------+------------+---------------------+----+-----+---+------+------+--------------+
only showing top 3 rows



In [23]:
df_Temperatur.show(3)

+-----------+------------+-------+--------+----+-----+---+------+------+----------------+
|STATIONS_ID|  MESS_DATUM|Temp_2m|Temp_5cm|Jahr|Monat|Tag|Stunde|Minute|              ID|
+-----------+------------+-------+--------+----+-----+---+------+------+----------------+
|       2171|201001010000|   -1.3|    -1.3|2010|   01| 01|    00|    00|2171201001010000|
|       2171|201001010010|   -1.4|    -1.3|2010|   01| 01|    00|    10|2171201001010010|
|       2171|201001010020|   -1.4|    -1.3|2010|   01| 01|    00|    20|2171201001010020|
+-----------+------------+-------+--------+----+-----+---+------+------+----------------+
only showing top 3 rows



In [24]:
#Bearbeiten von df_Solar --> Nur noch Werte in DF mit Monat = 01
#Quelle: https://stackoverflow.com/questions/52395986/remove-rows-from-dataframe-based-on-condition-in-pyspark
df_Solar = df_Solar.filter(df_Solar.Monat == '01')
df_Solar.count()

2451799

In [25]:
#Bearbeiten von df_Solar --> Nur noch Werte bei denen Sonnenscheindauer <> -999
df_Solar = df_Solar.filter(df_Solar.Sum_Sonnenscheindauer != -999)
df_Solar.count()

2325309

In [26]:
#Gruppieren der Daten nach StationsID und Jahr
#Quelle: https://sparkbyexamples.com/pyspark/pyspark-groupby-explained-with-example/
df_Solar_final = df_Solar.groupBy('STATIONS_ID','Jahr').agg(count('Sum_Sonnenscheindauer').alias('Anzahl_Eintraege'),(mean('Sum_Sonnenscheindauer')/0.06).alias('Anteil_Sonnenschein'))


In [27]:
#ID Column erstellen 
df_Solar_final = df_Solar_final.withColumn("ID",concat_ws('','STATIONS_ID','Jahr'))
df_Solar_final.show(3)

+-----------+----+----------------+-------------------+-------+
|STATIONS_ID|Jahr|Anzahl_Eintraege|Anteil_Sonnenschein|     ID|
+-----------+----+----------------+-------------------+-------+
|        131|2018|            4464|0.15684363576600255|1312018|
|         73|2014|            4464|0.12657183236103137| 732014|
|        131|2011|            4464|0.24299581331816217|1312011|
+-----------+----+----------------+-------------------+-------+
only showing top 3 rows



In [28]:
df_Temperatur = df_Temperatur.filter((df_Temperatur.Monat == '06') | (df_Temperatur.Monat == '07') | (df_Temperatur.Monat == '08'))
df_Temperatur.count()

2766987

In [29]:
df_Temperatur.show(3)

+-----------+------------+-------+--------+----+-----+---+------+------+----------------+
|STATIONS_ID|  MESS_DATUM|Temp_2m|Temp_5cm|Jahr|Monat|Tag|Stunde|Minute|              ID|
+-----------+------------+-------+--------+----+-----+---+------+------+----------------+
|       2171|201006010000|    9.9|     9.3|2010|   06| 01|    00|    00|2171201006010000|
|       2171|201006010010|    9.9|     9.2|2010|   06| 01|    00|    10|2171201006010010|
|       2171|201006010020|    9.8|     9.2|2010|   06| 01|    00|    20|2171201006010020|
+-----------+------------+-------+--------+----+-----+---+------+------+----------------+
only showing top 3 rows



In [30]:
df_Temperatur = df_Temperatur.filter((df_Temperatur.Temp_2m != -999)|(df_Temperatur.Temp_5cm != -999))
df_Temperatur.count()

2668938

In [31]:
df_Temperatur_final = df_Temperatur.groupBy('STATIONS_ID','Jahr').agg(count('Temp_2m').alias('Anzahl_Eintraege'),(mean('Temp_2m')).alias('Durchschnittstemperatur'))
df_Temperatur_final.show(3)

+-----------+----+----------------+-----------------------+
|STATIONS_ID|Jahr|Anzahl_Eintraege|Durchschnittstemperatur|
+-----------+----+----------------+-----------------------+
|       2171|2011|           13248|      16.63435235507249|
|       2252|2013|           13248|      17.43346920289854|
|       2252|2017|           13248|     17.966945954106286|
+-----------+----+----------------+-----------------------+
only showing top 3 rows



In [32]:
#ID Column erstellen 
df_Temperatur_final = df_Temperatur_final.withColumn("ID",concat_ws('','STATIONS_ID','Jahr'))
df_Temperatur_final.show(3)

+-----------+----+----------------+-----------------------+--------+
|STATIONS_ID|Jahr|Anzahl_Eintraege|Durchschnittstemperatur|      ID|
+-----------+----+----------------+-----------------------+--------+
|       2171|2011|           13248|      16.63435235507249|21712011|
|       2252|2013|           13248|      17.43346920289854|22522013|
|       2252|2017|           13248|     17.966945954106286|22522017|
+-----------+----+----------------+-----------------------+--------+
only showing top 3 rows



In [33]:
#Verbinden der beiden DF df_Temperature_final und df_Solar_Final
#Quelle: https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/
df_final = df_Solar_final.join(df_Temperatur_final,df_Solar_final.ID == df_Temperatur_final.ID,'left')
df_final.show(3)


+-----------+----+----------------+-------------------+-------+-----------+----+----------------+-----------------------+-------+
|STATIONS_ID|Jahr|Anzahl_Eintraege|Anteil_Sonnenschein|     ID|STATIONS_ID|Jahr|Anzahl_Eintraege|Durchschnittstemperatur|     ID|
+-----------+----+----------------+-------------------+-------+-----------+----+----------------+-----------------------+-------+
|        131|2011|            4464|0.24299581331816217|1312011|        131|2011|           13248|      16.98316727053144|1312011|
|         73|2014|            4464|0.12657183236103137| 732014|         73|2014|           13248|      17.78292572463777| 732014|
|         73|2011|            4464|0.20750074267939286| 732011|         73|2011|           13166|     17.856380069876913| 732011|
+-----------+----+----------------+-------------------+-------+-----------+----+----------------+-----------------------+-------+
only showing top 3 rows



In [34]:
df_final.count()

540

In [35]:
#Correlation zwischen Sonnenschein und Durschsnittstemperatur
df_final.stat.corr('Anteil_Sonnenschein','Durchschnittstemperatur')

-0.03054000433767041